# 다층신경망(MLP)

## 1) 기본 Package 설정

In [1]:
# 기본
import numpy as np
import matplotlib.pyplot as plt # Graph

# 데이터 가져오기
import pandas as pd
from sklearn import datasets

# 데이터 전처리
from sklearn.preprocessing import StandardScaler    # 연속 변수 표준화
from sklearn import preprocessing                   # 범주형 변수 수치화
from sklearn.preprocessing import LabelEncoder      # 범주형 변수 수치화

# 훈련/검증용 데이터 분리
from sklearn.model_selection import train_test_split    # 훈련과 테스트를 위한 데이터 분리

# 분류 모델
# from sklearn.tree import DecisionTreeClassifier         # 의사결정나무
# from sklearn.naive_bayes import GaussianNB            # 나이브 베이즈 분류
# from sklearn.neighbors import KNeighborsClassifier      # K-최근접 이웃
# from sklearn.ensemble import RandomForestClassifier   # 랜덤 포레스트
# from sklearn.linear_model import LogisticRegression     # 로지스틱 회귀분석
# from sklearn.svm import SVC                           # SVM(서포트벡터머신)
from sklearn.neural_network import MLPClassifier      # 다층 인공신경망
# from sklearn.ensemble import VotingClassifier           # 과반수 투표(Majority Voting) 
# from sklearn.ensemble import BaggingClassifier        # 배깅(Bagging) 
# from sklearn.ensemble import AdaBoostClassifier       # 부스팅(Boosting) 

# 모델 검정
from sklearn.metrics import confusion_matrix, classification_report # 정오분류표
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer  # 정확도, 민감도 등
from sklearn.metrics import roc_curve, roc_auc_score, auc   # ROC 곡선

# 최적화
from sklearn.model_selection import cross_validate, cross_val_score  # 교차 타당도
from sklearn.pipeline import make_pipeline  # 파이프라인 구축
from sklearn.model_selection import learning_curve, validation_curve # 학습곡선, 검증곡선
from sklearn.model_selection import GridSearchCV    # 하이퍼파라미터 튜닝

## 2) 데이터 가져오기

### 2-1) 데이터 프레임으로 저장
* csv 데이터를 dataframe으로 가져오기

In [2]:
train = pd.read_csv('../Data/train.csv')

# train data의 상위 5개 출력
train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,73,0


### 2-2) 자료구조 살펴보기

In [3]:
# null값 확인
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  awards_won?           54808 non-null  int64  
 11  avg_training_score    54808 non-null  int64  
 12  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 5.4+ MB
None


In [4]:
# 배열의 행, 열 갯수 확인
print(train.shape)

(54808, 13)


In [5]:
# column 값 확인
train.keys()

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'awards_won?', 'avg_training_score',
       'is_promoted'],
      dtype='object')

## 3) 데이터 전처리
### 3-1) ONE HOT ENCODING
- 범주형 변수는 먼저 숫자로 변경해주어야 한다.
- one-hot-encoding을 통해 string을 integer로 변환할 수 있다

In [6]:
train_df = pd.get_dummies(train)

In [7]:
# train_df data의 상위 5개 출력
train_df.head()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score,is_promoted,department_Analytics,department_Finance,...,region_region_8,region_region_9,education_Bachelor's,education_Below Secondary,education_Master's & above,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
0,65438,1,35,5.0,8,0,49,0,0,0,...,0,0,0,0,1,1,0,0,0,1
1,65141,1,30,5.0,4,0,60,0,0,0,...,0,0,1,0,0,0,1,1,0,0
2,7513,1,34,3.0,7,0,50,0,0,0,...,0,0,1,0,0,0,1,0,0,1
3,2542,2,39,1.0,10,0,50,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4,48945,1,45,3.0,2,0,73,0,0,0,...,0,0,1,0,0,0,1,1,0,0


### 3-2) null값 변경

In [8]:
# null값을 평균 값으로 대체
train_df['previous_year_rating'].fillna(train_df['previous_year_rating'].mean(),inplace=True)

In [9]:
# null값 반영 여부 확인
print(f'데이터 셋의 Null 값 개수:{train_df.isnull().sum().sum()}')

데이터 셋의 Null 값 개수:0


### 3-3) Data와 Target으로 분리
- 필요한 data만 추출
- data : `X`, taget : `y`로 분리

In [10]:
# data
X = train_df.drop(['employee_id','is_promoted'], axis=1)
X.head()

,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score,department_Analytics,department_Finance,department_HR,department_Legal,...,region_region_8,region_region_9,education_Bachelor's,education_Below Secondary,education_Master's & above,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
0,1,35,5.0,8,0,49,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
1,1,30,5.0,4,0,60,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
2,1,34,3.0,7,0,50,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
3,2,39,1.0,10,0,50,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4,1,45,3.0,2,0,73,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0


In [11]:
# target
y=train_df['is_promoted']
np.bincount(y)

array([50140,  4668])

## 4) 훈련 / 검증용 데이터 분할

### 4-1) 데이터 분리

In [12]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     test_size = 0.3,   # test set의 비율
                     random_state = 1,  # 무작위 시드 번호
                     stratify = y)      # 결과 레이블의 비율대로 분리

### 4-2) 표준화
- 표준화는 train data를 표준화한 포멧을 test data에도 wjrdyd
- train data를 표준화할 때 사용한 평균과 표준편차를 test data에 적용하여 변환


In [13]:
stdsc = StandardScaler()

X_train = stdsc.fit_transform(X_train)
X_test = stdsc.transform(X_test)